In [1]:
# imports & config

import os
import numpy as np
import nibabel as nb
import pandas as pd
import glob

output_folder = 'output'

In [2]:
# Read Freesurfer Features

def _vectorize_fs_surf(in_data_file):
    img = nb.load(in_data_file)
    in_data = img.get_data().squeeze()
    vectorized_data = in_data[np.newaxis, ...]
    assert vectorized_data.shape == (1, 2562), "%s shape does not look right: %s" % (in_data_file,
                                                                                     vectorized_data.shape)

    return vectorized_data

def _vectorize_fs_tab(in_data_file):
    df = pd.read_csv(in_data_file, index_col=0, delimiter='\t')
    vectorized_data = df.values
    assert vectorized_data.shape == (1, 66), "%s shape does not look right: %s" % (in_data_file,
                                                                                   vectorized_data.shape)
    return vectorized_data

def combine_surfs(lh_file, rh_file):
    lh_data = _vectorize_fs_surf(lh_file)
    rh_data = _vectorize_fs_surf(rh_file)
    return np.concatenate((lh_data, rh_data), 1)

def get_data(lh_thickness_file, rh_thickness_file, lh_area_file, rh_area_file, aseg_file):
    X = {}
    X["thickness"] = combine_surfs(lh_thickness_file, rh_thickness_file)
    X["area"] = combine_surfs(lh_area_file, rh_area_file)
    X["aseg"] = _vectorize_fs_tab(aseg_file)
    return (X)


In [3]:
aseg_files = []
lh_area_files = []
rh_area_files = []
lh_thickness_files = []
rh_thickness_files = []

for r,d,f in os.walk(output_folder):
    for file in f:
        if file.endswith('aseg') and not file.startswith('.'):
            aseg_files.append(os.path.join(r, file))
        if file.endswith('lh.area.mgh') and not file.startswith('.'):
            lh_area_files.append(os.path.join(r, file))
        if file.endswith('rh.area.mgh') and not file.startswith('.'):
            rh_area_files.append(os.path.join(r, file))
        if file.endswith('lh.thickness.mgh') and not file.startswith('.'):
            lh_thickness_files.append(os.path.join(r, file))
        if file.endswith('rh.thickness.mgh') and not file.startswith('.'):
            rh_thickness_files.append(os.path.join(r, file))

In [4]:
# traverse subjects

for i in range(len(aseg_files)):
    subj_data = get_data(lh_thickness_files[i], rh_thickness_files[i], lh_area_files[i], rh_area_files[i], aseg_files[i])
    if i==0:
        X = subj_data
    else:
        for k in X.keys():
            X[k] = np.concatenate((X[k], subj_data[k]))

In [5]:
# partition in training and test set

# train model